Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/img-classification-part1-training.png)

# Train and Deploy a Model on Azure Machine Learning service.

In this lab, you train the classifcation machine learning model on remote compute resources. You'll use the training and deployment workflow for Azure Machine Learning service (preview) in a Python Jupyter notebook.  

Learn how to:

> * Set up your development environment
> * Access and examine the data
> * Train a simple classification model on a remote cluster
> * Review training results, find and register the best model

## Set up your development environment

All the setup for your development work can be accomplished in a Python notebook.  Setup includes:

* Create an Experiment in an existing Workspace.
* Configure AutoML using AutoMLConfig.
* Importing Python packages
* Connecting to a workspace to enable communication between your local computer and remote resources
* Creating an experiment to track all your runs
* Creating a remote compute target to use for training

### Import packages

Import Python packages you need in this session. Also display the Azure Machine Learning SDK version.

In [1]:
import numpy as np

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.20.0


# Create the AMLS workspace...

## You will be asked to log into Azure and be given a code in the output message area to enter.

In [ ]:
from azureml.core import Workspace
ws = Workspace.create(name='docs-ws',
            subscription_id='c958680c-dc7a-403c-bb83-74f48dce46b3', 
            resource_group='docs-aml',
            create_resource_group = True,
            location='West US'
            )

If you got a Azure Machine Learning Workspace please use the cell above

In [2]:
from azureml.core import Workspace
ws = Workspace.get(name='docs-ws',
            subscription_id='c958680c-dc7a-403c-bb83-74f48dce46b3', 
            resource_group='docs-aml')

## Workspace Properties from Above:

Name | Description
---- | -----------
name            | A name you chose to call the workspace.  We'll use the value aready in the code.
subscription_id | The id of the subscription the workspace will be assigned to.  You can get this from the Azure portal.
resource_group  | A name you want all the Azure resourced creates for the workspace to be associated with.  Makes rsource management easier.
location | Azure data center location closest to you that support creation of AMLS workspaces.  

## To make reconnecting to this workspace in future notebooks easier, save the configuration setting using the code in the cell below.

In [3]:
# Create the configuration file.
ws.write_config(path='.', file_name='config.json')
print('Configuration saved.')

Configuration saved.


### Connect to workspace in future work...

In the future, we can use the code below to connect back to this workspace. 
The code creates a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `ws`.  You don't need to use this now since we are still connected from when we created the workspace but this will come in handy later.

In [4]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

docs-ws	westus	docs-aml	westus


### Create experiment

Create an experiment to track the runs in your workspace. A workspace can have muliple experiments. 

### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace the code will skip the creation process.

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 1)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D4_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

creating a new compute target...
Creating
Succeeded..................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-24T10:39:28.011000+00:00', 'errors': None, 'creationTime': '2021-01-24T10:37:35.915130+00:00', 'modifiedTime': '2021-01-24T10:37:52.959329+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D4_V2'}


You now have the necessary packages and compute resources to train a model in the cloud. 

## Verify you have the data

You already explored the data in the last lab.  You need to copy the data into the cloud so it can be accessed by your cloud training environment.  We saved the model training data to a csv file so all we have to do is load it.

### Display a few rows of data to make sure the load worked.

In [9]:
import pandas as pd
df_features = pd.read_csv(r'inputs/BikeFeatures.csv', sep=';')
df_features.head()

,BikeTypeCat,AgeBandCat,IncomeBandCat,MaritalStatusCat,OccupationCat,CommuteDistanceCat,GenderCat,ChildrenAtHomeCat,CountryRegionCodeCat,HouseOwnerFlagCat,EducationCat
0,1,5,1,1,1,2,1,0,1,1,0
1,0,3,0,1,2,0,0,1,3,0,2
2,0,5,0,1,3,2,0,0,5,1,2
3,1,5,1,0,1,1,1,0,5,1,1
4,0,3,1,1,3,4,0,0,0,0,0


Now you have an idea of what these images look like and the expected prediction outcome.

### Upload data to the cloud

Now make the data accessible remotely by uploading that data from your local machine into Azure so it can be accessed for remote training. The datastore is a convenient construct associated with your workspace for you to upload/download data, and interact with it from your remote compute targets. It is backed by Azure blob storage account.

The data file is uploaded into a directory named `dp100lab` at the root of the datastore.

In [10]:
import urllib.request
data_folder = os.getcwd() 
print(data_folder)

/home/atabordal/azureml-tutorial


In [11]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir=data_folder, target_path='dp100lab', overwrite=True, show_progress=True)

AzureBlob docswsstorage02aa6558ea0 azureml-blobstore-22e259ad-ce66-4b4d-8174-9422c5ed1ce0
Uploading an estimated of 57 files
Uploading /home/atabordal/azureml-tutorial/LICENSE
Uploaded /home/atabordal/azureml-tutorial/LICENSE, 1 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/README.md
Uploaded /home/atabordal/azureml-tutorial/README.md, 2 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/.git/hooks/applypatch-msg.sample
Uploaded /home/atabordal/azureml-tutorial/.git/hooks/applypatch-msg.sample, 3 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/.git/hooks/pre-commit.sample
Uploaded /home/atabordal/azureml-tutorial/.git/hooks/pre-commit.sample, 4 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/bikeFeatures.py
Uploaded /home/atabordal/azureml-tutorial/bikeFeatures.py, 5 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/azure-ml.y

Uploading /home/atabordal/azureml-tutorial/Train and Deploy a Model on Azure.ipynb
Uploaded /home/atabordal/azureml-tutorial/Train and Deploy a Model on Azure.ipynb, 49 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/.git/objects/68/db527fa50968db9cd0edd109a179dc004fcbbe
Uploaded /home/atabordal/azureml-tutorial/.git/objects/68/db527fa50968db9cd0edd109a179dc004fcbbe, 50 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/.git/objects/c2/279c68148e73507e6858ebd11d4a147ba5f4ca
Uploaded /home/atabordal/azureml-tutorial/.git/objects/c2/279c68148e73507e6858ebd11d4a147ba5f4ca, 51 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/.git/objects/7e/d6093c2dd61fb62b2739d886119ecc420d0bc8
Uploaded /home/atabordal/azureml-tutorial/.git/objects/7e/d6093c2dd61fb62b2739d886119ecc420d0bc8, 52 files out of an estimated total of 57
Uploading /home/atabordal/azureml-tutorial/.git/objects/2c/c3ccc29987323266b56403b

$AZUREML_DATAREFERENCE_859fd12286af40f69e45852d2b22db4c

## Train on a remote cluster

For this task, submit the job to the remote training cluster you set up earlier.  To submit a job you:
* Create a directory
* Create a training script
* Create an estimator object
* Submit the job 

### Create a directory

Create a directory to deliver the necessary code from your computer to the remote resource.

In [13]:
import os
script_folder = os.path.join(os.getcwd(), "dp100lab")
os.makedirs(script_folder, exist_ok=True)

In [14]:
# Confirm the df_feeatures one more time...
df_features.dtypes

BikeTypeCat             int64
AgeBandCat              int64
IncomeBandCat           int64
MaritalStatusCat        int64
OccupationCat           int64
CommuteDistanceCat      int64
GenderCat               int64
ChildrenAtHomeCat       int64
CountryRegionCodeCat    int64
HouseOwnerFlagCat       int64
EducationCat            int64
dtype: object

### Create a training script

To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train.py` in the directory you just created. 

## About training scripts...

To train a model in an Azure container, we need to get the model training script to the container. We start by saving the model training script to a Python script file, i.e. .py.  This will be uploaded to the Azure container later.  We don't need any exploratory code in this script, just what is needed to train the model.

# Clasiification training script below.

In [15]:
%%writefile $script_folder/train.py

import argparse
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

from azureml.core import Run
print('Libraries Imported')

# ***  Azure Machine Learning service specfic code starts... ***

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--maxdepth', type=float, dest='max_depth', default=14, help='max_depth')
args = parser.parse_args()


data_folder = args.data_folder
max_depth = args.max_depth

print('Data folder:', data_folder)

# get hold of the current run
run = Run.get_context()

# ***  Azure Machine Learning service specfic code ends. ***

# filepath = data_folder + '/BikeModelFeatures.csv'
filepath = os.path.join(data_folder, 'BikeFeatures.csv')

df_features = pd.read_csv(filepath, sep=';')

# load train and test set into numpy arrays

X_train_all , X_test_all = train_test_split(df_features.values,test_size=0.2)      
#test_size=0.5(whole_data)

# Column 0 has the value we want to predict
X_train_all[:,0]

y_train = X_train_all[:,0]

y_test = X_test_all[:,0]

X_train_all.shape

X_train = X_train_all[:,1:11]
X_train.shape

X_test = X_test_all[:,1:11]
X_test

# Set the max_depth model hyperparameter to = max_depth which is the parameter value we created in the Azure ML service specific code above, i.e. , max_depth = max_depth 
# print('trainng RandomForestClassifier...')
#classifier = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', random_state = 42, max_depth = max_depth)
classifier =  AdaBoostClassifier(DecisionTreeClassifier(min_samples_leaf=10, max_depth = 20), n_estimators= 100, learning_rate=0.2)
classifier.fit(X_train, y_train)
print('Classifier Model Trained.')


# Predict using the test data...
print('Running the test dataset through...')
y_predtest = classifier.predict(X_test)
print('Test dataset scored.')

# calculate accuracy on the prediction
acc = np.average(y_predtest == y_test)
print('Accuracy is', acc)


# ***  Azure Machine Learning service specfic code starts... ***
run.log('data_dir', data_folder)
run.log('accuracy', np.float(acc))

os.makedirs('outputs', exist_ok=True)

# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=classifier, filename='outputs/biketypemodel.pkl')

# ***  Azure Machine Learning service specfic code ends. ***

Writing /home/atabordal/azureml-tutorial/dp100lab/train.py


Notice how the script gets data and saves models:

+ The training script reads an argument to find the directory containing the data.  When you submit the job later, you point to the datastore for this argument:
`parser.add_argument('--data-folder', type=str, dest='data_folder', help='data directory mounting point')`


+ The training script saves your model into a directory named outputs. <br/>
`joblib.dump(value=clf, filename='outputs/biketypemodel.pkl')`<br/>
Anything written in this directory is automatically uploaded into your workspace. You'll access your model from this directory later in the tutorial.

### Create an estimator

An estimator object is used to submit the run.  Create your estimator by running the following code to define:

* The name of the estimator object, `est`
* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The compute target.  In this case you will use the AmlCompute you created
* The training script name, train.py
* Parameters required from the training script 
* Python packages needed for training

In this tutorial, this target is AmlCompute. All files in the script folder are uploaded into the cluster nodes for execution. The data_folder is set to use the datastore (`ds.path('dp100lab').as_mount()`).

In [17]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.path('dp100lab').as_mount(),
    '--maxdepth': 12
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script='train.py',
                conda_packages=['scikit-learn','pandas'])

print('Executed')

'Estimator' is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or an Azure ML curated environment.


Executed


In [21]:
# create or load an experiment
experiment_name = 'dp100labexperiment'
experiment = Experiment(workspace=ws, name=experiment_name)
print('Experiment created.')

Experiment created.


In [25]:
from azureml.core import ScriptRunConfig, Environment

# create or retrieve an environment
env = Environment.get(ws, name='MyEnvironment')
# configure and submit your training run
config = ScriptRunConfig(source_directory=script_folder,
                            script='train.py',
                            arguments=script_params,
                            compute_target=compute_target,
                            environment=env)
script_run = experiment.submit(config)

Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No definitions exist for environment MyEnvironment",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null
  },
  "correlation": {
    "operation": "c9d51460bd441440bd29a56c4ae37489",
    "request": "868c37cadefcf445"
  },
  "environment": "westus",
  "location": "westus",
  "time": "2021-01-24T11:07:15.7498339+00:00",
  "componentName": "environment-management"
}

In [18]:
est  # take a look at the est object...

In [19]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=5, 
                                               tags={"data": "Bike Features",  "method" : "sklearn"}, 
                                               description='Classification model sklearn')

### Submit the job to the cluster

Run the experiment by submitting the estimator object. And you can navigate to Azure portal to monitor the run.

In [20]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
dp100labexperiment,dp100labexperiment_1611485423_2eafbd8b,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#  We can see what built-in datastores are available with this code below...
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type)

Since the call is asynchronous, it returns a **Preparing** or **Running** state as soon as the job is started.

## Monitor a remote run

In total, the first run takes **approximately 10 minutes**. But for subsequent runs, as long as the dependencies (`conda_packages` parameter in the above estimator constructor) don't change, the same image is reused and hence the container start up time is much faster.

Here is what's happening while you wait:

- **Image creation**: A Docker image is created matching the Python environment specified by the estimator. The image is built and stored in the ACR (Azure Container Registry) associated with your workspace. Image creation and uploading takes **about 5 minutes**. 

  This stage happens once for each Python environment since the container is cached for subsequent runs.  During image creation, logs are streamed to the run history. You can monitor the image creation progress using these logs.

- **Scaling**: If the remote cluster requires more nodes to execute the run than currently available, additional nodes are added automatically. Scaling typically takes **about 5 minutes.**

- **Running**: In this stage, the necessary scripts and files are sent to the compute target, then data stores are mounted/copied, then the entry_script is run. While the job is running, stdout and the files in the ./logs directory are streamed to the run history. You can monitor the run's progress using these logs.

- **Post-Processing**: The ./outputs directory of the run is copied over to the run history in your workspace so you can access these results.


You can check the progress of a running job in multiple ways. This tutorial uses a Jupyter widget as well as a `wait_for_completion` method. 

### Jupyter widget

Watch the progress of the run with a Jupyter widget.  Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

By the way, if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

### Get log results upon completion

Model training happens in the background. You can use `wait_for_completion` to block and wait until the model has completed training before running more code. 

In [ ]:
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=False) 

### Display run results

You now have a model trained on a remote cluster.  Retrieve all the metrics logged during the run, including the accuracy of the model:

In [ ]:
print(run.get_metrics())

In the next tutorial you will explore this model in more detail.

## Register model

The last step in the training script wrote the file `outputs/biketypemodel.pkl` in a directory named `outputs` in the VM of the cluster where the job is executed. `outputs` is a special directory in that all content in this  directory is automatically uploaded to your workspace.  This content appears in the run record in the experiment under your workspace. Hence, the model file is now also available in your workspace.

You can see files associated with that run.

In [ ]:
print(run.get_file_names())

Register the model in the workspace so that you (or other collaborators) can later query, examine, and deploy this model.

In [ ]:
# register model 
model = run.register_model(model_name='biketypemodel', model_path='outputs/biketypemodel.pkl')
print(model.name, model.id, model.version, sep='\t')

_______________________

# Lab 2: Part B Deploy the model

Now, you're ready to deploy the model as a web service in Azure Container Instances (ACI). A web service is an image, in this case a Docker image, that encapsulates the scoring logic and the model itself.

In this part of the lab, you use Azure Machine Learning service (Preview) to:

Set up your testing environment
Retrieve the model from your workspace
Test the model locally
Deploy the model to ACI
Test the deployed model

ACI is not ideal for production deployments, but it is great for testing and understanding the workflow. For scalable production deployments, consider using AKS.

### Retrieve the model

You registered a model in your workspace previously. Assuming you needed to come back later to deploy the model, load this workspace and download the model to your local directory.

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model=Model(ws, 'biketypemodel')
model.download(target_dir='.', exist_ok=True)
import os 
# verify the downloaded model file
os.stat('./biketypemodel.pkl')

print('Complete')

## Deploy as web service

Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI. 

To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('biketypemodel')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

### Create environment file

Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `scikit-learn` and `azureml-sdk`.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

bikemodelenv = CondaDependencies()
bikemodelenv.add_conda_package("scikit-learn")

with open("bikemodelenv.yml","w") as f:
    f.write(bikemodelenv.serialize_to_string())
    
print('Complete')    

Review the content of the `myenv.yml` file.

In [ ]:
with open("bikemodelenv.yml","r") as f:
    print(f.read())
    
print('Complete')    

### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "biketype",  "method" : "sklearn"}, 
                                               description='Predict biketype with sklearn')

print('Complete')

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="bikemodelenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='sklearn-biketype-svc2',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

print('Complete')

Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(service.scoring_uri)

## Test deployed service

Earlier you scored all the test data with the local version of the model. Now, you can test the deployed model by just passing the test data we used earlier, X_test.  We're doing this in the interest of time as it will serve us well just to test the deployed service.  In a real use case, you might want to specially prepare some input data.  

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

1. Use the SDK's `run` API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

1. Print the returned predictions and plot them along with the input images. Red font and inverse image (white on black) is used to highlight the misclassified samples. 


In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
X_train_all , X_test_all = train_test_split(df_features.values,test_size=0.2)       #test_size=0.5(whole_data)
X_train_all[:,0]
y_train = X_train_all[:,0]
y_test = X_test_all[:,0]
X_train_all.shape
X_train = X_train_all[:,1:11]
X_train.shape
X_test = X_test_all[:,1:11]
X_test



In [ ]:
df_BikeTest = pd.read_csv('BikeFeatures.csv', sep=';')
y_test = df_BikeTest['BikeTypeCat']
df_BikeTest.drop(columns=['BikeTypeCat'], inplace=True)
X_test = df_BikeTest.values
X_test.shape

In [ ]:
import json


test_samples = json.dumps({"data": X_test.tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)

print('Complete')

### Display the first few values to see that the call worked.

In [ ]:
result[0:5]

You can also send raw HTTP request to test the web service.  Again, we are using the X_test data for convenience.

In [ ]:
import requests
import json

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test)-1)
input_data = "{\"data\": [" + str(list(X_test[random_index])) + "]}"


headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
#print("input data:", input_data)
print("label:", y_test[random_index])
print("prediction:", resp.text)

print('Complete')

## Clean up resources

To keep the resource group and workspace for other tutorials and exploration, you can delete only the ACI deployment using this API call:

In [ ]:
# Uncomment the line below and run the code to clean up the resources.

# service.delete()